In [4]:
from threading import Thread
from multiprocessing import Process
import multiprocessing
import numpy as np
import subprocess
import signal
import time
import os
import sys

In [ ]:
def start_stress(num_cores, time):
    print("Stressing ",num_cores," cores for ",time," seconds")
    t= str(time)+'s'
    cmd= 'stress '+'-c '+str(num_cores)+' -t '+ t
    pid_stress = subprocess.Popen(['stress', '-c', str(num_cores), "-t", t],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE, preexec_fn=os.setsid)
    return pid_stress

In [ ]:
def add_data():
    global model_num
    global name_dataSet
    global lst_header
    global lst_Wt
    global lst_Ct
    global lst_TAc
    global lst_Ac
    global lst_Pr
    global lst_Re
    
    name_file= ""
    
    if(model_num == 0):
        lst_header[0]= "Logistic Regression"
        name_file= "_raspLR"
    elif(model_num == 1):
        lst_header[0]= "Support Vector Machine"
        name_file= "_raspSVM"
    elif(model_num == 2):
        lst_header[0]= "Gradient Boosting"
        name_file= "_raspGB"
    elif(model_num == 3):
        lst_header[0]= "Random Forest"
        name_file= "_raspRF"
        
    data= np.array([lst_header, lst_Wt, lst_Ct, lst_TAc, lst_Ac, lst_Pr, lst_Re])
    
    name_file=  name_dataSet+name_file+".csv"  
    
    np.savetxt(name_file,data,delimiter=",",fmt='%s')

In [ ]:
def get_WallT(line, lst_Wt):
    
    wall_time= line.replace("real\t","").split("m")
    minutes= float(wall_time[0])*60
    sec= float(wall_time[1].replace("s",""))
    sec= round(sec, 2)
    
    wallTime= minutes+sec
        
    lst_Wt.append(wallTime)

In [ ]:
def get_CpuT(user_line, sys_line, lst_Ct):
    
    user_time= user_line.split("\t")[1]
    user_time= user_time.split("m")
    
    sys_time= sys_line.split("\t")[1]
    sys_time= sys_time.split("m")
    
    minutes= int(user_time[0])+int(sys_time[0])
    minutes= minutes*60
    sec= float(user_time[1].replace("s",""))+float(sys_time[1].replace("s",""))
    sec= round(sec,2)
    
    cpuTime= minutes+sec
    
    lst_Ct.append(cpuTime)

In [ ]:
def get_APR(output, lst_Ac, lst_Pr, lst_Re):
    output=output.split("Trainning Accuracy")[1]#para quitar caracteres extranos
    output=output.split("\n")
    
    tac_output= output[0].split(":")[1].split("%")[0].replace(" ","")
    ac_output= output[1].split(":")[1].split("%")[0].replace(" ","")
    pre_output= output[2].split(":")[1].split("%")[0].replace(" ","")
    rec_output= output[3].split(":")[1].split("%")[0].replace(" ","")
    
    lst_TAc.append(tac_output)
    lst_Ac.append(ac_output)
    lst_Pr.append(pre_output)
    lst_Re.append(rec_output)

In [ ]:
def get_data(m_name, output, times):
    global num_cores
    global list_cpus
    global name_dataset
    global lst_header
    global lst_Wt
    global lst_Ct
    global lst_TAc
    global lst_Ac
    global lst_Pr
    global lst_Re
    
    output= output.decode("utf-8")
    times= times.decode("utf-8")
    
    if(num_cores== list_cpus[0]):
        lst_header= ["","Idle"]
        lst_Wt= ["Wall time"]
        lst_Ct= ["Cpu time"]
        lst_TAc= ["Trainning Accuracy"]
        lst_Ac= ["Accuracy"]
        lst_Pr= ["Precision"]
        lst_Re= ["Recall"]
        
    elif(num_cores > 0):
        lst_header.append((str(num_cores)+" stress"))
    
    lines= times.split("\n")
    
    get_WallT(lines[1], lst_Wt)
    
    get_CpuT(lines[2], lines[3], lst_Ct)
    
    get_APR(output, lst_Ac, lst_Pr, lst_Re)
    
    
    if(num_cores==list_cpus[-1]):
        add_data()

In [ ]:
def model(model_num):
    global dataSet
    global num_cores
    global name_dataset
    
        
    m_name= ""
    
    if model_num == 0:
        m_name= "Logistic Regression"
        print("Executing ",m_name)
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/lRegression.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
        
    if model_num == 1:
        m_name= "Maquinas de soporte vectorial"
        print("Executing ",m_name)
        m_name= "\n"+m_name
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/svm.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()

    if model_num == 2:
        m_name= "Gradient Boosting"
        print("Executing ",m_name)
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/gBoosting.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
    
    if model_num == 3:
        m_name= "Random Forest"
        print("Executing ",m_name)
        output, times = subprocess.Popen(['bash', '-c', 'time "$@" 2>&1', '_',"ipython", './Modelos/rForest_raspberry.ipynb', dataSet],
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
        
        
    get_data(m_name, output,times)

In [5]:
program= sys.argv[0].split("/")

if(len(program) > 1):
    program= program[-1]
    
if (len(sys.argv) != 2):
    error=""
    error=error+"Error: wrong number of arguments, execute as: \nipython "+program+" dataset_name"
    sys.exit(error)

SystemExit: Error: wrong number of arguments, execute as: 
python /home/nuriadj/anaconda3/envs/tfg/lib/python3.9/site-packages/ipykernel_launcher.py dataset_name

/home/nuriadj/anaconda3/envs/tfg/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#Hacer una version que ejecute de seguido los modelos solo (sin stress)
t= 200
list_cpus= [0,1,2,4]
dataSet= sys.argv[1]
name_dataSet= dataSet.split("/")[-1].split(".csv")[0]

lst_header= []
lst_Wt= []
lst_Ct= []
lst_TAc= []
lst_Ac= []
lst_Pr= []
lst_Re= []

for i in range(4):
    for j in list_cpus:
        num_cores= j
        
        if(j > 0):
            pid_stress= start_stress(j, t)
        else:
            print("\nCpu Idle")
            
        model_num= i    
        model(model_num)
        
        if j > 0:
            os.killpg(os.getpgid(pid_stress.pid), signal.SIGKILL)
        
        print("Sleeping for 5 sec...\n")
        time.sleep(5)